In [1009]:
###############################
# Imports
###############################
import torch

# Tensordict modules
from tensordict.nn import TensorDictModule
from tensordict.nn.distributions import NormalParamExtractor
from torch import multiprocessing
from tensordict import TensorDictBase
from tensordict.nn import TensorDictModule, TensorDictSequential


# Data collection
from torchrl.collectors import SyncDataCollector
from torchrl.data.replay_buffers import ReplayBuffer
from torchrl.data.replay_buffers.samplers import SamplerWithoutReplacement
from torchrl.data.replay_buffers.storages import LazyTensorStorage

# Env
from torchrl.envs import RewardSum, TransformedEnv
from torchrl.envs.libs.vmas import VmasEnv
from torchrl.envs.utils import check_env_specs

# Multi-agent network
from torchrl.modules import MultiAgentMLP, ProbabilisticActor, TanhNormal

# Loss
from torchrl.objectives import ClipPPOLoss, ValueEstimators

# Utils
torch.manual_seed(0)
from matplotlib import pyplot as plt
from tqdm import tqdm
import copy
import tempfile

# CPU/GPU
is_fork = multiprocessing.get_start_method() == "fork"
device = (torch.device(0) if torch.cuda.is_available() and not is_fork else torch.device("cpu"))
vmas_device = device  # The device where the simulator is run (VMAS can run on GPU)
print(device)

cuda:0


In [1010]:
###############################
# Hyperparams
###############################

# Sampling (frames per batch)
frames_per_batch = 6_000  # Number of team frames collected per training iteration
n_iters = 10  # Number of sampling and training iterations
total_frames = frames_per_batch * n_iters

# Training (batch size)
num_epochs = 30  # Number of optimization steps per training iteration
minibatch_size = 400  # Size of the mini-batches in each optimization step
lr = 3e-4  # Learning rate
max_grad_norm = 1.0  # Maximum norm for the gradients

# PPO
clip_epsilon = 0.2  # clip value for PPO loss
gamma = 0.99  # discount factor
lmbda = 0.9  # lambda for generalised advantage estimation
entropy_eps = 1e-4  # coefficient of the entropy term in the PPO loss

In [1011]:
###############################
# Environment Setup
###############################
max_steps = 100  # Steps before 'done' is automatically set.
num_vmas_envs = (frames_per_batch // max_steps)  # Leverage batch simulation. 
scenario_name = "simple_tag"

# Num pursuers/evaders
n_pursuers = 2
n_evaders = 1
n_obstacles = 2
render = "human" # '' or 'human'

base_env = VmasEnv(
    scenario=scenario_name,
    num_envs=num_vmas_envs,
    continuous_actions=True,
    max_steps=max_steps,
    device=vmas_device,

    num_good_agents=n_evaders,
    num_adversaries=n_pursuers,
    num_landmarks=n_obstacles,
    render_mode=render,
)

c:\Users\dfc49\miniconda3\envs\simple_tag-gpu\Lib\site-packages\vmas\simulator\utils.py:317: UserWarning: Scenario kwargs: {'render_mode': 'human'} passed but not used by the scenario. This will turn into an error in future versions.
  warnings.warn(


In [1012]:
###############################
# Printing environment keys
###############################
# All specs have leading shape (num_vmas_envs, n_agents) except for 'done.'
# print("action_spec:", env.full_action_spec)
# print("reward_spec:", env.full_reward_spec)
# print("done_spec:", env.full_done_spec)
# print("observation_spec:", env.observation_spec)

# print("action_keys:", env.action_keys)
print("reward_keys:", env.reward_keys)
# print("done_keys:", env.done_keys)
print(base_env.full_action_spec[('adversary', 'action')])

reward_keys: [('adversary', 'reward'), ('agent', 'reward')]
BoundedContinuous(
    shape=torch.Size([60, 2, 2]),
    space=ContinuousBox(
        low=Tensor(shape=torch.Size([60, 2, 2]), device=cuda:0, dtype=torch.float32, contiguous=True),
        high=Tensor(shape=torch.Size([60, 2, 2]), device=cuda:0, dtype=torch.float32, contiguous=True)),
    device=cuda:0,
    dtype=torch.float32,
    domain=continuous)


In [1013]:
###############################
# Modify environment
# Group pursuer rewards
###############################
env = TransformedEnv(
    base_env,
    RewardSum(
        in_keys=base_env.reward_keys,
        reset_keys=["_reset"] * len(base_env.group_map.keys()),
    )
)
check_env_specs(env) # Check env setup

2024-12-13 04:47:49,513 [torchrl][INFO] check_env_specs succeeded!


In [1014]:
# ###############################
# # Rollout
# ############################### 
# with torch.no_grad():
#    env.rollout(
#        max_steps=max_steps,
#        #policy=policy,
#        callback=lambda env, _: env.render(),
#        auto_cast_to_device=True,
#        break_when_any_done=False,
#    )

In [1015]:
###############################
# Policy Network
# Returns policy_modules[group], policies[group]
###############################

# 1) Create MLP agents for adversaries
policy_modules = {}
for group, agents in env.group_map.items():
    share_parameters_policy = True

    policy_net = torch.nn.Sequential(
        MultiAgentMLP(
            n_agent_inputs=env.observation_spec[group, "observation"].shape[-1],  # n_obs_per_agent
            n_agent_outputs=2 * env.full_action_spec[group, "action"].shape[-1],  # 2 * n_actions_per_agents
            n_agents=len(agents),
            centralised=False,  # the policies are decentralised (ie each agent will act from its observation)
            share_params=share_parameters_policy,
            device=device,
            depth=2,
            num_cells=256,
            activation_class=torch.nn.Tanh,
        ), #NormalParamExtractor is REQUIRED
        NormalParamExtractor(),  # this will just separate the last dimension into two outputs: a loc and a non-negative scale
    )

    # 2) Wrap NN -> TensorDictModule
    policy_module = TensorDictModule(
        policy_net,
        in_keys=[(group, "observation")],
        out_keys=[(group, "loc"), (group, "scale")], # TODO: maybe outkeys needs to be 1 param
    )

    policy_modules[group] = policy_module # Group = 'adversary' or 'agent'




# 3) Create ProbabilisticActor from policy networks
policies = {}
for group, _agents in env.group_map.items():
    policy = ProbabilisticActor(
        module=policy_modules[group],
        spec=env.full_action_spec[group, "action"],
        in_keys=[(group, "loc"), (group, "scale")],
        out_keys=[(group, "action")],
        distribution_class=TanhNormal,
        distribution_kwargs={
            "low": env.full_action_spec[group, "action"].space.low,
            "high": env.full_action_spec[group, "action"].space.high,
        },
        return_log_prob=True,
        log_prob_key=(group, "sample_log_prob"),
    )  # we'll need the log-prob for the PPO loss
    policies[group] = policy


In [1016]:
###############################
# Critic Network
# Returns critics[group]
###############################
share_parameters_critic = True
mappo = True  # IPPO if False

critics = {}
for group, agents in env.group_map.items():

    cat_module = TensorDictModule(
        lambda obs, action: torch.cat([obs, action], dim=-1),
        in_keys=[(group, "observation"), (group, "action")],
        out_keys=[(group, "obs_action")],
    )

    # TODO: do i need cat module
    critic_net = MultiAgentMLP(
        n_agent_inputs=env.observation_spec[group, "observation"].shape[-1] + env.full_action_spec[group, "action"].shape[-1],
        n_agent_outputs=1,  # 1 value per agent
        n_agents=len(agents),
        centralised=mappo,
        share_params=share_parameters_critic,
        device=device,
        depth=2,
        num_cells=256,
        activation_class=torch.nn.Tanh,
    )

    critic_module = TensorDictModule(
        module=critic_net,
        in_keys=[(group, "obs_action")],
        out_keys=[(group, "state_value")],
    )

    critics[group] = TensorDictSequential(
        cat_module, critic_module
    )

In [1017]:
# Run policy example
reset_td = env.reset()
for group, _agents in env.group_map.items():
    print(
        f"Running value and policy for group '{group}':",
        critics[group](policies[group](reset_td)),
    )

Running value and policy for group 'adversary': TensorDict(
    fields={
        adversary: TensorDict(
            fields={
                action: Tensor(shape=torch.Size([60, 2, 2]), device=cuda:0, dtype=torch.float32, is_shared=True),
                episode_reward: Tensor(shape=torch.Size([60, 2, 1]), device=cuda:0, dtype=torch.float32, is_shared=True),
                loc: Tensor(shape=torch.Size([60, 2, 2]), device=cuda:0, dtype=torch.float32, is_shared=True),
                obs_action: Tensor(shape=torch.Size([60, 2, 16]), device=cuda:0, dtype=torch.float32, is_shared=True),
                observation: Tensor(shape=torch.Size([60, 2, 14]), device=cuda:0, dtype=torch.float32, is_shared=True),
                sample_log_prob: Tensor(shape=torch.Size([60, 2]), device=cuda:0, dtype=torch.float32, is_shared=True),
                scale: Tensor(shape=torch.Size([60, 2, 2]), device=cuda:0, dtype=torch.float32, is_shared=True),
                state_value: Tensor(shape=torch.Size([60

In [1018]:
###############################
# Data Collector
###############################
print(policies['adversary'])
collector = SyncDataCollector(
    env,
    TensorDictSequential([policies['adversary'], policies['agent']]),
    device=vmas_device,
    storing_device=device,
    frames_per_batch=frames_per_batch,
    total_frames=total_frames,
)
print(collector)

###############################
# Replay Buffer
###############################
replay_buffers = {}
for group, _agents in env.group_map.items():
    replay_buffer = ReplayBuffer(
        # TODO: lazytensor or lazymemmap
        storage=LazyTensorStorage(frames_per_batch, device=device),  # We will store up to memory_size multi-agent transitions
        sampler=SamplerWithoutReplacement(), 
        batch_size=minibatch_size,  # Sample this size
    )
    replay_buffers[group] = replay_buffer

ProbabilisticActor(
    module=ModuleList(
      (0): TensorDictModule(
          module=Sequential(
            (0): MultiAgentMLP(
                MLP(
                  (0): Linear(in_features=14, out_features=256, bias=True)
                  (1): Tanh()
                  (2): Linear(in_features=256, out_features=256, bias=True)
                  (3): Tanh()
                  (4): Linear(in_features=256, out_features=4, bias=True)
                ),
                n_agents=2,
                share_params=True,
                centralized=False,
                agent_dim=-2)
            (1): NormalParamExtractor(
              (scale_mapping): biased_softplus()
            )
          ),
          device=cuda:0,
          in_keys=[('adversary', 'observation')],
          out_keys=[('adversary', 'loc'), ('adversary', 'scale')])
      (1): SafeProbabilisticModule()
    ),
    device=cuda:0,
    in_keys=[('adversary', 'observation')],
    out_keys=[('adversary', 'loc'), ('adversary', 

c:\Users\dfc49\miniconda3\envs\simple_tag-gpu\Lib\site-packages\torchrl\collectors\collectors.py:194: UserWarning: A policy device was provided but no parameter/buffer could be found in the policy. Casting to policy_device is therefore impossible. The collector will trust that the devices match. To suppress this warning, set `trust_policy=True` when building the collector.
  warnings.warn(


TypeError: 'list' object is not callable

In [958]:
###############################
# Loss Function
###############################
losses = {}
GAEs = {}
optimisers = {}
for group, _agents in env.group_map.items():
    loss_module = ClipPPOLoss(
        actor_network=policies[group],  # Use the non-explorative policies
        critic_network=critics[group],
        clip_epsilon=clip_epsilon,
        entropy_coef=entropy_eps,
        normalize_advantage=False,  # Important to avoid normalizing across the agent dimension
    )
    loss_module.set_keys(
        reward=(group, "reward"),
        action=(group, "action"),
        sample_log_prob=(group, "sample_log_prob"),
        value=(group, "state_value"),
        # These last 2 keys will be expanded to match the reward shape
        done=(group, "done"),
        terminated=(group, "terminated"),
    )


    loss_module.make_value_estimator(
        ValueEstimators.GAE, # TD0 VS GAE
        gamma=gamma, 
        lmbda=lmbda
    ) 
    losses[group] = loss_module

    GAE = loss_module.value_estimator # we build GAE TODO: do for each agent/adv?
    GAEs[group] = GAE

    optimisers[group] = torch.optim.Adam(loss_module.parameters(), lr)

In [963]:
# FIX DATA COLLECTOR

pbar = tqdm(total=n_iters, desc="episode_reward_mean = 0")

episode_reward_mean_list = []
for group in train_group_map.keys():
    print(collector['adversary'])
    for tensordict_data in collector[group]:
        pass
        # tensordict_data.set(
        #     ("next", group, "done"),
        #     tensordict_data.get(("next", "done"))
        #     .unsqueeze(-1)
        #     .expand(tensordict_data.get_item_shape(("next", (group, "reward")))),
        # )
        # tensordict_data.set(
        #     ("next", group, "terminated"),
        #     tensordict_data.get(("next", "terminated"))
        #     .unsqueeze(-1)
        #     .expand(tensordict_data.get_item_shape(("next", (group, "reward")))),
        # )
        # We need to expand the done and terminated to match the reward shape (this is expected by the value estimator)

        # with torch.no_grad():
        #     GAE(
        #         tensordict_data,
        #         params=loss_module.critic_network_params,
        #         target_params=loss_module.target_critic_network_params,
        #     )  # Compute GAE and add it to the data

        # data_view = tensordict_data.reshape(-1)  # Flatten the batch size to shuffle data
        # replay_buffer.extend(data_view)

        # for _ in range(num_epochs):
        #     for _ in range(frames_per_batch // minibatch_size):
        #         subdata = replay_buffer.sample()
        #         loss_vals = loss_module(subdata)

        #         loss_value = (
        #             loss_vals["loss_objective"]
        #             + loss_vals["loss_critic"]
        #             + loss_vals["loss_entropy"]
        #         )

        #         loss_value.backward()

        #         torch.nn.utils.clip_grad_norm_(
        #             loss_module.parameters(), max_grad_norm
        #         )  # Optional

        #         optim.step()
        #         optim.zero_grad()

        # collector.update_policy_weights_()

        # # Logging
        # done = tensordict_data.get(("next", group, "done"))
        # episode_reward_mean = (
        #     tensordict_data.get(("next", group, "episode_reward"))[done].mean().item()
        # )
        # episode_reward_mean_list.append(episode_reward_mean)
        # pbar.set_description(f"episode_reward_mean = {episode_reward_mean}", refresh=False)
        # pbar.update()


episode_reward_mean = 0:   0%|          | 0/10 [00:11<?, ?it/s]

SyncDataCollector(
    env=TransformedEnv(
        env=VmasEnv(num_envs=60, n_agents=3, batch_size=torch.Size([60]), device=cuda:0) (scenario=simple_tag),
        transform=RewardSum(keys=[('adversary', 'reward'), ('agent', 'reward')])),
    policy=ProbabilisticActor(
        module=ModuleList(
          (0): TensorDictModule(
              module=Sequential(
                (0): MultiAgentMLP(
                    MLP(
                      (0): Linear(in_features=14, out_features=256, bias=True)
                      (1): Tanh()
                      (2): Linear(in_features=256, out_features=256, bias=True)
                      (3): Tanh()
                      (4): Linear(in_features=256, out_features=4, bias=True)
                    ),
                    n_agents=2,
                    share_params=True,
                    centralized=False,
                    agent_dim=-2)
                (1): NormalParamExtractor(
                  (scale_mapping): biased_softplus()
         


c:\Users\dfc49\miniconda3\envs\simple_tag-gpu\Lib\site-packages\tensordict\base.py:5028: DeprecationWarning: The entry you have queried with `get` is not present in the tensordict. Currently, this raises an exception. To align with `dict.get`, this behaviour will be changed in v0.7 and a `None` value will be returned instead (no error will be raised). To suppress this warning and use the new behaviour (recommended), call `tensordict.set_get_defaults_to_none(True)` or set the env variable `export TD_GET_DEFAULTS_TO_NONE='1'`. To suppress this warning and keep the old behaviour, call `tensordict.set_get_defaults_to_none(False)` or set the env variable `export TD_GET_DEFAULTS_TO_NONE='0'`.
  warnings.warn(


KeyError: 'key "action" not found in TensorDict with keys [\'episode_reward\', \'observation\']'